In [7]:
!pip install pykafka confluent_kafka avro

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 750.1 kB/s eta 0:00:00 0:00:01m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 6.0 MB/s eta 0:00:0000:0100:01
  Created wheel for avro: filename=avro-1.11.3-py2.py3-none-any.whl size=123913 sha256=07504323dd27f3054c965f96e1f97c71515f5ee81e17c82c38623371438aa329
  Stored in directory: /home/jovyan/.cache/pip/wheels/a8/7c/a4/fa31e47be300f6e6036f57769474de0ba54f8c6e8e2d8b7547
Successfully built avro


In [6]:
from pykafka import KafkaClient

client = KafkaClient(hosts="broker:29092", broker_version="1.0.0")

#print(client.topics)


topic = client.topics['training']

with topic.get_sync_producer() as producer:
    for i in range(4):
        value = 'test message ' + str(i ** 2)
        producer.produce(value.encode('utf-8'))

In [5]:
!set -x \
    && curl -fSL "https://www.apache.org/dist/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz" -o /tmp/hadoop.tar.gz \
    && curl -fSL "$HADOOP_URL.asc" -o /tmp/hadoop.tar.gz.asc \
    && tar -xvf /tmp/hadoop.tar.gz -C /opt/ \
    && rm /tmp/hadoop.tar.gz*

+ curl -fSL https://www.apache.org/dist/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz -o /tmp/hadoop.tar.gz
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   283  100   283    0     0   1693      0 --:--:-- --:--:-- --:--:--  1704
100  696M  100  696M    0     0  7685k      0  0:01:32  0:01:32 --:--:-- 8700k0:00:42  0:01:06 8642k
+ curl -fSL .asc -o /tmp/hadoop.tar.gz.asc
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: .asc


In [ ]:
!ln -s /opt/hadoop-3.3.6/etc/hadoop /etc/hadoop \
    && mkdir /opt/hadoop-3.3.6/logs \
    && mkdir /hadoop-data \
    && rm -Rf /opt/hadoop-3.3.6/share/doc/hadoop

In [7]:
consumer_topic = client.topics['training']
consumer = consumer_topic.get_simple_consumer()
for message in consumer:
    if message is not None:
        print(message.offset, message.value)

0 b'test message 0'
1 b'test message 1'
2 b'test message 4'
3 b'test message 9'
4 b'test message 0'
5 b'test message 1'
6 b'test message 4'
7 b'test message 9'
8 b'test message 0'
9 b'test message 1'
10 b'test message 4'
11 b'test message 9'


KeyboardInterrupt: 

In [19]:
import json

topic = client.topics['topic_schema']

with topic.get_sync_producer() as producer:
    #for i in range(4):

    value = {
        "name":"Gökhan",
        "age":49
    }        

    # Mesajı JSON formatına dönüştürün
    message_json = json.dumps(value)
    
    # Mesajı bytes'a dönüştürün
    message_bytes = message_json.encode('utf-8')
    #value = 'test message ' + str(i ** 2)
    producer.produce(message_bytes)

KeyError: 87

In [2]:

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .config("spark.streaming.stopGracefullyOnShutdown", True) \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0') \
    .config("spark.sql.shuffle.partitions", 4) \
    .master("local").getOrCreate()
spark

In [8]:
import avro
from avro import schema, datafile, io
from confluent_kafka.avro import AvroProducer


producer_config =  {
                    'bootstrap.servers': 'broker:29092',
                    'schema.registry.url': 'http://schema-registry:8081'
                    }

topic_producer = AvroProducer(producer_config, default_value_schema=avro.schema.parse(open("sample-schema.avsc", "rb").read()))


value = {
        "name":"Gökhan",
        "age":49
        } 
topic_producer.produce(topic="topic_schema",value=value)
topic_producer.flush()


/tmp/ipykernel_820/377846958.py:11: DeprecationWarning: AvroProducer has been deprecated. Use AvroSerializer instead.
  topic_producer = AvroProducer(producer_config, default_value_schema=avro.schema.parse(open("sample-schema.avsc", "rb").read()))


0

In [33]:
streaming_df = spark.readStream\
    .format("kafka") \
    .option("kafka.bootstrap.servers", "broker:29092") \
    .option("subscribe", "topic_schema") \
    .option("startingOffsets", "earliest") \
    .load()
streaming_df.printSchema()

json_df = streaming_df.selectExpr("cast(value as string) as value", "timestamp")
json_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, StructField, StructType, ArrayType, LongType

json_schema = StructType(
    [StructField('name', StringType(), True), \
     StructField('age', LongType(), True)])

json_expanded_df = json_df.withColumn("value", from_json(json_df["value"], json_schema)).select(col("value.age"),col("value.name"),col("timestamp")) 
json_expanded_df.printSchema()

flattened_df = json_expanded_df.selectExpr("name", "age","timestamp")
flattened_df.printSchema()
#agg_df = flattened_df.agg(avg("age").alias("age_temp"))

flattened_df.withWatermark("timestamp", "10 seconds").writeStream \
    .format("csv") \
    .option("path", "output.csv") \
    .option("append", "true") \
    .option("checkpointLocation", "checkpoint") \
    .start()
spark.streams.awaitAnyTermination()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)

